In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

def shortest_distance(x1, y1, a, b, c):  
            d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b)) 
            return(d)

def get_intersections(x0, y0, r0, x1, y1, r1):
            # circle 1: (x0, y0), radius r0
            # circle 2: (x1, y1), radius r1

            d=math.sqrt((x1-x0)**2 + (y1-y0)**2)

            # non intersecting
            if d > r0 + r1 :
                return (x0,y0,x1,y1)
            # One circle within other
            if d < abs(r0-r1):
                return (x0,y0,x1,y1)
            # coincident circles
            if d == 0 and r0 == r1:
                return (x0,y0,x1,y1)
            else:
                a=(r0**2-r1**2+d**2)/(2*d)
                h=math.sqrt(r0**2-a**2)
                x2=x0+a*(x1-x0)/d   
                y2=y0+a*(y1-y0)/d   
                x3=x2+h*(y1-y0)/d     
                y3=y2-h*(x1-x0)/d 

                x4=x2-h*(y1-y0)/d
                y4=y2+h*(x1-x0)/d

                return (x3, y3, x4, y4)

anchorx=[]
anchory=[]
sensorx=[]
sensory=[]
s=["red","blue","yellow","green"]
N = 5
M = 10
n = 1.7 
C = -42
all = M
sum=0
sum1=0
minDist=1000
shortestDist=1000
errorRate=0.025
transmissionRange=200
realDistance = [[0 for x in range(N)] for y in range(M)]
errorDistance = [[0 for x in range(N)] for y in range(M)]
rssi = [[0 for x in range(N)] for y in range(M)]
rssi1000 = [[0 for x in range(N)] for y in range(M)]
rssiall=[[0 for x in range(N)] for y in range(M*10)]
rssipredicted=[[0 for x in range(N)] for y in range(M)]
predicted_distance=[[0 for x in range(N)] for y in range(M)]
errorr = [[0 for x in range(10)] for y in range(M)]
error1000 = [[0 for x in range(10)] for y in range(M)]
#generate the coordinates of the anchor nodes randomly
for i in range (N):
    x1 = np.random.randint(200)
    y1 = np.random.randint(200)
    anchorx.append(x1)
    anchory.append(y1)
#generate the coordinates of the sensor nodes randomly
for i in range (M):
    x2 = np.random.randint(200)
    y2 = np.random.randint(200)
    sensorx.append(x2)
    sensory.append(y2)
#drawing a figure that shows the deployment of the nodes
plt.rcParams["figure.figsize"] = (15,15)
plt.scatter(anchorx, anchory,70,"red",">")    
plt.scatter(sensorx, sensory,70,"blue",".")    

#calcule the real distance of each sensor node with every anchor node
for i in range (M):
    anchorCount = 0
    for x in range (N):
        distance = math.sqrt((sensorx[i]-anchorx[x])**2 + (sensory[i]-anchory[x])**2)
        realDistance[i][x] = distance
        
count=0      
for h in range(10):
    rssi=np.zeros((M,N))
    for k in range(1000):
        for i in range (M):
            for x in range (N):
                error = random.uniform(-errorRate*transmissionRange,errorRate*transmissionRange)
                errorDistance[i][x] = realDistance[i][x] + error

        #Estimating RSSI from distance
        for i in range (M):
            for x in range (N):
                rssi[i][x] = rssi[i][x] -10 * math.log10(abs(errorDistance[i][x]))* n + C
        plt.ylim([0,200])
        plt.xlim([0,200])
        
        for w in range(M):
            a=get_intersections(anchorx[0],anchory[0],errorDistance[w][0],anchorx[1],anchory[1],errorDistance[w][1])
            b=get_intersections(anchorx[0],anchory[0],errorDistance[w][0],anchorx[2],anchory[2],errorDistance[w][2])
            c=get_intersections(anchorx[2],anchory[2],errorDistance[w][2],anchorx[1],anchory[1],errorDistance[w][1])
            xint=[a[0],a[2],b[0],b[2],c[0],c[2]]
            yint=[a[1],a[3],b[1],b[3],c[1],c[3]]

            for i in range (6):
                for x in range (5):
                    Dist= math.sqrt((xint[i]-xint[x+1])**2 + (yint[i]-yint[x+1])**2)
                    if(Dist!=0):
                        if(Dist<minDist):
                            minDist=Dist
                            minx1=xint[i]
                            miny1=yint[i]
                            minx2=xint[x+1]
                            miny2=yint[x+1]
            minDist=1000

            if (minx2==minx1):
                a=0
                c=miny2
            else:
                a=(miny2-miny1)/(minx2-minx1)
                c=miny2-a*minx2

            for i in range(6):
                d =shortest_distance(xint[i],yint[i],a,-1,c)
                if(0<d<shortestDist):
                    shortestDist=d
                    minx3=xint[i]
                    miny3=yint[i]
            shortestDist=1000
            centroidx=(minx1+minx2+minx3)/3
            centroidy=(miny1+miny2+miny3)/3
            sum+=math.sqrt((sensorx[w]-centroidx)**2 + (sensory[w]-centroidy)**2)
            
            errorr[w][h]= errorr[w][h] + math.sqrt((sensorx[w]-centroidx)**2 + (sensory[w]-centroidy)**2)
        

    for i in range (M):
        for j in range(N):
            rssi1000[i][j]=rssi[i][j]/1000
    ##print(rssi1000)
    
    
    for i in range(all-M,all):
        for j in range(N):
            rssiall[i][j]=rssi1000[i%M][j]
    all+=M
    
    count+=1
    if(count<8):         
        errorRate+=0.025
    if(count==8):
        errorRate+=0.6
 
    
for k in range(M):
    for l in range (10):
        error1000[k][l]=errorr[k][l]/1000
##print(error1000)


from sklearn.cluster import KMeans
for i in range(M):
    first=0
    second=0
    X=[rssiall[i],rssiall[i+M],rssiall[i+(2*M)],rssiall[i+(3*M)],rssiall[i+(4*M)],rssiall[i+(5*M)],rssiall[i+(6*M)],rssiall[i+(7*M)],rssiall[i+(8*M)],rssiall[i+(9*M)]]
    kmeans=KMeans(n_clusters=2,random_state=0).fit(X)
    a=kmeans.labels_
    b=kmeans.cluster_centers_
    for j in range(h+1):
        if(a[j]==0):
            first+=1
        else:
            second+=1
    if(first>second):
        for k in range (N):
            rssipredicted[i][k]=b[0][k]
    
    else:
        for l in range (N):
            rssipredicted[i][l]=b[1][l]

for i in range (M):
    for j in range (N):
        predicted_distance[i][j] = 10**((rssipredicted[i][j]-C)/(-10*n))
##print (predicted_distance)

for w in range(M):
        a=get_intersections(anchorx[0],anchory[0],predicted_distance[w][0],anchorx[1],anchory[1],predicted_distance[w][1])
        b=get_intersections(anchorx[0],anchory[0],predicted_distance[w][0],anchorx[2],anchory[2],predicted_distance[w][2])
        c=get_intersections(anchorx[2],anchory[2],predicted_distance[w][2],anchorx[1],anchory[1],predicted_distance[w][1])
        xint=[a[0],a[2],b[0],b[2],c[0],c[2]]
        yint=[a[1],a[3],b[1],b[3],c[1],c[3]]

        for i in range (6):
            for x in range (5):
                Dist= math.sqrt((xint[i]-xint[x+1])**2 + (yint[i]-yint[x+1])**2)
                if(Dist!=0):
                    if(Dist<minDist):
                        minDist=Dist
                        minx1=xint[i]
                        miny1=yint[i]
                        minx2=xint[x+1]
                        miny2=yint[x+1]
        minDist=1000
        if (minx2==minx1):
            a=0
            c=miny2
        else:
            a=(miny2-miny1)/(minx2-minx1)
            c=miny2-a*minx2
        for i in range(6):
            d =shortest_distance(xint[i],yint[i],a,-1,c)
            if(0<d<shortestDist):
                shortestDist=d
                minx3=xint[i]
                miny3=yint[i]
        shortestDist=1000
        centroidx1=(minx1+minx2+minx3)/3
        centroidy1=(miny1+miny2+miny3)/3
        if(centroidx1>200):
            centroidx1=200
        if(centroidy1>200):
            centroidy1=200
        if(centroidx1<0):
            centroidx1=0
        if(centroidy1<0):
            centroidy1=0
        x_coordinates1=[centroidx1,sensorx[w]]
        y_coordinates1=[centroidy1,sensory[w]]
        
        sum1+=math.sqrt((sensorx[w]-centroidx1)**2 + (sensory[w]-centroidy1)**2)

        plt.plot(x_coordinates1, y_coordinates1)
        plt.scatter(centroidx1,centroidy1, 50,"orange","*")

accuracy=(1-((sum/(M*10000))/transmissionRange))*100
accuracy1=(1-((sum1/M)/transmissionRange))*100

print("The accuracy without machine learning is "+ str("{:.2f}".format(accuracy))+"%")
print("The accuracy using K-Means is "+str("{:.2f}".format(accuracy1))+"%")